In [1]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


BERT 개념 이해

https://moondol-ai.tistory.com/463


In [2]:
!pip install ratsnlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 KB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.5/582.5 KB 55.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 KB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.4/512.4 KB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 90.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 50.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=30b3728e6eedd1087975bf285e2a3925b682722ca4ae8780e7d5ccc3ae9ef25c
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c1

# 토크나이저 초기화 
bert(kcbert-base) 모델이 쓰는 토크나이저를 선언합니다.

In [3]:
# 오픈소스 transformer in huggingface
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    "beomi/kcbert-base",
    do_lower_case=False,
)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

# 모델 초기화
bert 모델을 읽어드립니다.

In [4]:
# 오픈소스에서 config를 읽어들인다. 
from transformers import BertConfig, BertModel
pretrained_model_config = BertConfig.from_pretrained(
    "beomi/kcbert-base"
)
model = BertModel.from_pretrained(
    "beomi/kcbert-base",
    config=pretrained_model_config,
)

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


pretrained_model_config의 내용을 확인합니다.

In [5]:
# 블록(레이어) 수는 12개, 헤드의 수는 12개 , 어휘 집합의 크기는 3만개 등 정보를 확인 
pretrained_model_config

BertConfig {
  "_name_or_path": "beomi/kcbert-base",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 300,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.10.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30000
}

# 모델 입력값 만들기
문장 2개를 모델 입력값으로 만들어보겠습니다.

In [7]:
sentences = ["안녕하세요", "하이!"]
features = tokenizer(
    sentences,
    max_length=10,
    padding="max_length",
    truncation=True,
)

`features`의 내용을 확인
- 두 개의 입력 문장 각각에 대해 워드피스 토큰화를 수행한 뒤 이를 토큰 인덱스로 변환한 결과가 input_ids입니다. BERT 모델은 문장 시작에 CLS, 끝에 SEP라는 스페셜 토큰을 추가하기 때문에 문장 두 개 모두 앞뒤에 이들 토큰에 대응하는 인덱스 2, 3이 덧붙여져 있음을 볼 수 있다.

- 토큰 최대 길이(max_length)를 10으로 설정하고, 토큰 길이가 이보다 짧으면 최대 길이에 맞게 패딩(0)을 주고(padding="max_length"), 길면 자르는(truncation=True) 것으로 설정해 두었기 때문에 input_ids의 길이는 두 문장 모두 10인걸 확인


In [8]:
features.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [9]:
features['input_ids']

[[2, 19017, 8482, 3, 0, 0, 0, 0, 0, 0], [2, 15830, 5, 3, 0, 0, 0, 0, 0, 0]]

In [10]:
features['attention_mask']

[[1, 1, 1, 1, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 0, 0, 0, 0, 0, 0]]

In [11]:
features['token_type_ids']

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

## BERT 임베딩 추출

위에서 만든 `features`를 파이토치 텐서(tensor)로 변환
- 파이토치 모델의 입력값 자료형은 파이토치에서 제공하는 텐서(tensor)여야 합니다. 따라서 코드3에서 만든 파이썬 리스트(list) 형태의 features를 텐서로 변환


In [13]:
import torch
features = {k: torch.tensor(v) for k, v in features.items()}

BERT 모델 실행
 - features를 입력 계산합니다.

In [15]:
outputs = model(**features)

자연어를 벡터로 바꾼 결과를 임베딩(embedding) 또는 리프레젠테이션(representation)이라고 한다. 안녕하세요, 하이!라는 문장은 단어 수준의 벡터 시퀀스로, 문장 수준의 벡터로 변환할 수 있다. 전자를 단어 수준 임베딩(리프레젠테이션), 후자를 문장 수준 임베딩(리프레젠테이션)이라고 부른다.


BERT 마지막 레이어의 단어 수준 벡터들을 확인합니다

In [16]:
print(outputs.last_hidden_state.shape) # 문장 두 개에 속한 각각의 토큰(최대 길이 10) 을 768차원짜리의 벡터로 변환했다는 의미
outputs.last_hidden_state

torch.Size([2, 10, 768])


tensor([[[-0.6969, -0.8248,  1.7512,  ..., -0.3732,  0.7399,  1.1907],
         [-1.4803, -0.4398,  0.9444,  ..., -0.7405, -0.0211,  1.3064],
         [-1.4299, -0.5033, -0.2069,  ...,  0.1285, -0.2611,  1.6057],
         ...,
         [-1.4406,  0.3431,  1.4043,  ..., -0.0565,  0.8450, -0.2170],
         [-1.3625, -0.2404,  1.1757,  ...,  0.8876, -0.1054,  0.0734],
         [-1.4244,  0.1518,  1.2920,  ...,  0.0245,  0.7572,  0.0080]],

        [[ 0.9371, -1.4749,  1.7351,  ..., -0.3426,  0.8050,  0.4031],
         [ 1.6095, -1.7269,  2.7936,  ...,  0.3100, -0.4787, -1.2491],
         [ 0.4861, -0.4569,  0.5712,  ..., -0.1769,  1.1253, -0.2756],
         ...,
         [ 1.2362, -0.6181,  2.0906,  ...,  1.3677,  0.8132, -0.2742],
         [ 0.5409, -0.9652,  1.6237,  ...,  1.2395,  0.9185,  0.1782],
         [ 1.9001, -0.5859,  3.0156,  ...,  1.4967,  0.1924, -0.4448]]],
       grad_fn=<NativeLayerNormBackward0>)

BERT 마지막 레이어의 문서 수준 벡터를 확인
- 문장 두 개를 각각 768차원짜리의 벡터로 변환
- 문서 분류 과제 같이 문장 전체를 벡터 하나로 변환한 뒤 이 벡터에 어떤 계산을 수행하는 태스크에 활용




In [18]:
print(outputs.pooler_output.shape)
outputs.pooler_output

torch.Size([2, 768])


tensor([[-0.1594,  0.0547,  0.1101,  ...,  0.2684,  0.1596, -0.9828],
        [-0.9221,  0.2969, -0.0110,  ...,  0.4291,  0.0311, -0.9955]],
       grad_fn=<TanhBackward0>)

태스크 모듈 만들기
- 파인튜닝을 수행하기 위해서는 단어 혹은 문장 수준 임베딩 위에 태스크를 수행하기 위한 작은 모듈을 추가
- 다운스트림 데이터로 프리트레인 마친 BERT와 그 위의 작은 모듈을 포함한 전체 모델을 업데이트하는 과정
- 어떤 모듈을 사용할지는 다운스트림 태스크별로 조금씩 다르다.
- 문장 벡터 활용 : 문서 분류 등
  - 문장을 워드피스(wordpiece)로 토큰화한 뒤 앞뒤에 문장 시작과 끝을 알리는 스페셜 토큰 CLS와 SEP를 각각 추가한 뒤 BERT에 입력
  - 이후 BERT 모델의 마지막 블록(레이어)의 출력 가운데 CLS에 해당하는 벡터를 추출. 트랜스포머 인코더 블록에서는 모든 단어가 서로 영향을 끼치기 때문에 마지막 블록 CLS 벡터는 문장 전체(이 영화 재미없네요)의 의미가 벡터 하나로 응집된 것이라고 할 수 있다.
  - 이렇게 뽑은 CLS 벡터에 작은 모듈을 하나 추가해, 그 출력이 미리 정해 놓은 범주(예컨대 긍정, 중립, 부정)가 될 확률이 되도록 한다.
  - BERT와 그 위에 쌓은 작은 모듈을 포함한 전체 모델의 출력이 정답 레이블과 최대한 같아지도록 모델 전체를 업데이트
- 단어 벡터 활용 : 개체명 인식 등
  - 문서 분류는 마지막 블록의 CLS 벡터만을 사용하는 반면, 개체명 인식 같은 과제에서는 마지막 블록의 모든 단어 벡터를 활용
  - 문서 분류 때와 동일한 방식으로 입력값을 만들고 BERT의 마지막 레이어까지 계산을 수행.
  - BERT 모델의 마지막 블록(레이어)의 출력은 문장 내 모든 단어에 해당하는 벡터들의 시퀀스가 된다.
  - 이렇게 뽑은 단어 벡터들 위에 작은 모듈을 각각 추가해, 그 출력이 각 개체명 범주(기관명, 인명, 지명 등)가 될 확률이 되도록 한다.
  - 학습 과정에서는 BERT와 그 위에 쌓은 각각의 작은 모듈을 포함한 전체 모델의 출력이 정답 레이블과 최대한 같아지도록 모델 전체를 업데이트
